# SQL - Книги

Ваша компания купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Задания:
* Посчитайте, сколько книг вышло после 1 января 2000 года;
* Для каждой книги посчитайте количество обзоров и среднюю оценку;
* Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
* Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
* Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd 
from sqlalchemy import create_engine 
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
 db_config['pwd'], 
 db_config['host'], 
 db_config['port'], 
 db_config['db']) 
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
pd.io.sql.read_sql('''SELECT * FROM books LIMIT 5''', con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Содержит данные о книгах:

* `book_id` — идентификатор книги;
* `author_id` — идентификатор автора;
* `title` — название книги;
* `num_pages` — количество страниц;
* `publication_date` — дата публикации книги;
* `publisher_id` — идентификатор издателя.

In [3]:
pd.io.sql.read_sql('''SELECT * FROM  authors LIMIT 5''', con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Содержит данные об авторах:

* `author_id` — идентификатор автора;
* `author` — имя автора.

In [4]:
pd.io.sql.read_sql('''SELECT * FROM  publishers LIMIT 5''', con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Содержит данные об издательствах:

* `publisher_id` — идентификатор издательства;
* `publisher` — название издательства.

In [5]:
pd.io.sql.read_sql('''SELECT * FROM  ratings LIMIT 5''', con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Содержит данные о пользовательских оценках книг:

* `rating_id` — идентификатор оценки;
* `book_id` — идентификатор книги;
* `username` — имя пользователя, оставившего оценку;
* `rating` — оценка книги.

In [6]:
pd.io.sql.read_sql('''SELECT * FROM  reviews LIMIT 5''', con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Содержит данные о пользовательских обзорах на книги:

* `review_id` — идентификатор обзора;
* `book_id` — идентификатор книги;
* `username` — имя пользователя, написавшего обзор;
* `text` — текст обзора

In [7]:
# Посчитайте, сколько книг вышло после 1 января 2000 года
pd.io.sql.read_sql(
    '''SELECT COUNT(DISTINCT book_id) 
         FROM books 
        WHERE publication_date > '2000-01-01'
    '''
, con = engine)

,count
0,819


819 книг вышло после 1 января 2000 года.

In [8]:
# Для каждой книги посчитайте количество обзоров и среднюю оценку
pd.io.sql.read_sql(
    '''SELECT b.title,
              b.book_id,
              c_r.count_rev AS reviews,
              a_r.avg_rat AS avg_rating
         FROM books AS b
              LEFT JOIN 
                       (SELECT r.book_id,
                               AVG(r.rating) AS avg_rat
                          FROM ratings AS r
                      GROUP BY r.book_id) AS a_r ON a_r.book_id = b.book_id
              LEFT JOIN 
                       (SELECT rev.book_id,
                               COUNT(rev.review_id) AS count_rev
                          FROM reviews AS rev
                      GROUP BY rev.book_id) AS c_r ON c_r.book_id = b.book_id
     ORDER BY avg_rating DESC;
    '''
, con = engine)

,title,book_id,reviews,avg_rating
0,Pop Goes the Weasel (Alex Cross #5),518,2.0,5.00
1,The Ghost Map: The Story of London's Most Terr...,732,2.0,5.00
2,In the Hand of the Goddess (Song of the Liones...,347,2.0,5.00
3,Tai-Pan (Asian Saga #2),610,2.0,5.00
4,How to Be a Domestic Goddess: Baking and the A...,330,1.0,5.00
...,...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,915,3.0,2.25
996,Junky,371,2.0,2.00
997,His Excellency: George Washington,316,2.0,2.00
998,Drowning Ruth,202,3.0,2.00


In [9]:
# Определите издательство, которое выпустило наибольшее число книг толще 50 страниц
pd.io.sql.read_sql(
    '''SELECT p.publisher,
              COUNT(b.title) AS count_titles
         FROM books AS b
         JOIN publishers AS p ON b.publisher_id = p.publisher_id
        WHERE b.num_pages > 50
     GROUP BY p.publisher
     ORDER BY count_titles DESC
        LIMIT 5;
    '''
, con = engine)

,publisher,count_titles
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


Penguin Books выпустило наибольшее число книг толще 50 страниц - 42 штуки.

In [10]:
# Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками
pd.io.sql.read_sql(
    '''SELECT au_r.author,
              AVG(au_r.avg_of_rating) AS avg_rating
       FROM
           (SELECT a.author,
                   AVG(r.rating) AS avg_of_rating
              FROM books AS b
              JOIN authors AS a ON b.author_id = a.author_id
              JOIN ratings AS r ON b.book_id = r.book_id
          GROUP BY b.book_id, a.author
      HAVING COUNT(r.rating) >= 50) AS au_r
     GROUP BY au_r.author
     ORDER BY avg_rating DESC; 
    '''
, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Автор с самой высокой средней оценкой - J.K. Rowling/Mary GrandPré - 4,283844.

In [11]:
# Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок
pd.io.sql.read_sql(
    '''SELECT AVG(over_50_rev.counts_rev)
         FROM
             (SELECT username,
                     COUNT(review_id) AS counts_rev
                FROM reviews
               WHERE username IN 
                     (SELECT username
                        FROM ratings
                    GROUP BY username
                HAVING COUNT(rating_id) > 50)
                    GROUP BY username) AS over_50_rev;
    '''
, con = engine)

,avg
0,24.333333


Среднее количество обзоров от пользователей - 24,33.

# Вывод

1. 819 книг вышло после 1 января 2000 года.
2. Penguin Books выпустило наибольшее число книг толще 50 страниц - 42 штуки.
3. Автор с самой высокой средней оценкой - J.K. Rowling/Mary GrandPré - 4,283844.
4. Среднее количество обзоров от пользователей - 24,33.